Do sada smo diskutovali problem loše uslovljenih matrica i njihovu numeričku nestabilnost. U ovoj svesci upoznaćemo neke tehnike koje mogu da poprave korišćenje ovakvih matrica u problemima najmanjih kvadrata i, specifično, linearne regresije.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

### Tihonovljeva regularizacija

Posmatrajmo sistem $Ax = b$. Ukoliko je matrica $A^TA$ loše uslovljena kod problema najmanjih kvadrata umesto rešavanja osnovnog problema $$min_{x} \|A\cdot x - b\|^2$$ rešava se problem $$min_{x} \|A\cdot x - b\|^2 + \alpha ||x||^2$$. Parametar $\alpha$ se zove regularizacioni parametar, a samo rešenje $x$ ima formu $x=(A^{T}A + \alpha I)^{-1}A^{T}b$. 

In [2]:
A = np.array([[1, 2], [2, 3.999]])
b = np.array([4, 7.999]).T

In [3]:
A

array([[1.   , 2.   ],
       [2.   , 3.999]])

In [4]:
def lstsq_solution(A, b):
    return np.linalg.inv(A.T.dot(A)).dot(A.T).dot(b)

In [5]:
x = lstsq_solution(A, b)

In [6]:
x

array([2.00000012, 0.99999998])

In [7]:
b1 = np.copy(b)
b1[0] =  b1[0] + 0.001 
b1[1] =  b1[1] - 0.001 

In [8]:
x1 = lstsq_solution(A, b1)

In [9]:
x1

array([-3.99899991,  4.        ])

## Grebena regresija

U kontekstu linearne regresije pri rešavanju sistema $A\beta = Y$ metodom najmanjih kvadrata, gde se minimizuje vrednost izraza $||Y - \beta A||_2^2$ i čije je rešenje dato u obliku $\beta=(A^{T}A)^{-1}A^{T}Y$, može se dogoditi da matrica $A^{T}A$ nije invertibilina ili loše uslovljena. Da bi se ovo nadomestilo, umesto pomenute minimizacije, vrši se minimizacija $||Y - \beta A||_2^2 + \alpha ||\beta||_2^2$, za pogodno izabran koeficijent $\alpha$. Umesto linearne, tada se govori o grebenoj regresiji, a izraz $\alpha ||\beta||_2^2$ se naziva regularizacioni element.

Dalje ćemo na osnovu podataka sadržanih u datoteci *hitters.csv* koja sadrži statistike o performansama bejzbol igrača, predvideti visine njihovih plata. 

In [10]:
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection

Najpre učitajmo podatke i izvucimo osnovne informacije o njima.

Zaključujemo da postoji 322 unosa i 21 kolona. Od toga, jedino kod plata bejzbol igrača imamo neke nedostajuće vrednosti. Pošto nam takvi unosi nisu relevantni, obrisaćemo ih (ukupno ima takvih 59 unosa).

Sada nam je preostalo 263 unosa, bez nedostajućih vrednosti. Kolona *Salary* je vrednost koju tražimo. Za vektor nezavisnih promenljivih, pored kolone *Salary*, potrebno je odbaciti sve one kolone koje nemaju numeričku vrednost koja je od značaja za visinu plate. Među tim vrednostima su *Player*, *League*, *Division* i *NewLeague*.

Vektor nezavisnih promenljivih $X$ i vektor zavisnih promenljivih $Y$ ćemo podeliti na skup za treniranje, skup za validaciju i skup za testiranje. Ideja je da na skupu za testiranje testiramo performanse naših modela, da validacioni skup iskoristimo za ocenu optimalne vrednosti regularizacionog parametra, a da skup za treniranje iskoristimo za samo treniranje modela. Razmera podele ovih skupova će biti 60%: 20%: 20%. Ovakva podela skup će pomoći da se izbegne problem preprilagodjavanja (engl. *overfitting*).

<img src='assets/train_validation_test.png'>

Ispišimo sada osnovne statistike skupa X_train.

Vidimo da su numeričke vrednosti podataka neuravnotežene. Za neke je maksimalna vrednosti nekoliko desetina, a za druge i do nekoliko hiljada. Da bi se model bolje ponašao, podatke ćemo standardizovati. U tu svrhu, koristićemo klasu `StandardScaler`.

In [11]:
from sklearn import preprocessing

Pogledajmo kako izgledaju standardizovani podaci na primeru prvih nekoliko unosa:

Sada možemo da primenimo model linearne regresije, a probaćemo i sa grebenom regresijom za, na primer, $\alpha = 0.5$ i $\alpha = 10^{10}$ (probamo sa malim i velikim vrednostima). U sva tri slučaja, izračunaćemo MSE. Za grebenu regresiju se koristi klasa `Ridge`. Koristi se na sličan način kao `LinearRegression`, sa dodatnim argumentom za vrednost parametra.

Vidimo da u prvom slučaju grebena regresija daje bolji rezultat od linearne, a u drugom nešto lošiji. Ovo nas motiviše da razmotrimo razne vrednosti parametra $\alpha$. Ispitaćemo 100 tačaka, koji će imati vrednost između $10^{-3}$ i $10^{10}$. Dakle, razmatramo različite redove veličine.

Za svaki parametar posebno je kreiran model grebene regresije, i izračunata MSE, koja je dodata u niz svih grešaka. Nacrtajmo grafik grešaka u zavisnosti od $\alpha$. Zbog načina izbora parametra $\alpha$ biće korišćena logaritamska skala. Kao najbolje odabrani model grebene regresije, koristićemo onaj za koji je greška najmanja.